In [1]:
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np

# Set up the plotting template
pio.templates["custom"] = pio.templates["plotly"]
pio.templates["custom"].layout.margin = dict(l=0, r=0, t=50, b=0)
pio.templates.default = "custom"

## Correlation

Before diving in to the DFT, here is some background on correlation. Correlation is very similar to convolution. Given two input signals, it produces a third signal. The algorithm to find the correlation of two signals similarly involves multiplying the input signal with the target signal and summing the points to create $y[i]$, the output signal.

The purpose of correlation is to detect a known waveform within another waveform. An example would be using an echo-location system (e.g. radar) sending a pulse with a known waveform to detect a moving object. The reflected signal would be shrouded with noise with the shape of our original pulse baked in somewhere. We would use correlation to detect where that known waveform lies within the received signal.

In [2]:
# Target signal
t = np.zeros(20)
t[:7] = np.array([0, 1.0, 0.8, 0.6, 0.4, 0.2, 0.0])

# Received signal with added noise
x = 0.15*np.random.rand(80) - 0.15/2
x[17:24] += t[:7]
x *= 0.2

fig = make_subplots(rows=1, cols=2, subplot_titles=("Transmit signal t[n]", "Received signal x[n]"))
fig.add_trace(go.Scatter(y=t,
                         marker=dict(size=10, symbol="square"),
                         mode="lines+markers"),
              row=1, col=1)
fig.add_trace(go.Scatter(y=x,
                         marker=dict(size=10, symbol="square"),
                         mode="lines+markers"),
              row=1, col=2)
fig.update_layout(showlegend=False)
fig.update_xaxes(title_text="Sample number (or time)")
fig.update_yaxes(title_text="Amplitude")
fig.show()

When the target signal $t[n]$ is correlated with the received signal $x[n]$, the result is called the **cross-correlation signal**, $y[n]$. The amplitude of each point in the cross-correlation signal is a measure of how similar the received signal is to the target signal at that particular location. This means that at a given point in $y[n]$, a peak will occur when the shape of $x[n]$ lines up with the shape of $t[n]$. The values in that location will be maximized, indication a detection of the target waveform. This does not necessarily mean the cross-correlation signal will look like the target signal, only that there will be a peak where $t[n]$ is detected in $x[n]$ $^{[1]}$.

Calculating the cross-correlation signal is almost identical to convolution, except that the target signal is not swapped left-for-right, as the impulse response $h[n]$ is in correlation.

In [3]:
y = np.zeros(len(x))
for i in range(len(x)):
    for j in range(len(t)):
        if i+j >= len(x):
            continue
        y[i] += t[j] * x[i+j]

fig = px.scatter(y)
fig.update_layout(title="Cross-correlation signal y[n]", showlegend=False)
fig.update_traces(marker=dict(size=10, symbol="square"), mode="lines+markers")
fig.update_xaxes(title_text="Sample number (or time)")
fig.update_yaxes(title_text="Amplitude")
fig.show()

There is still noise in the cross-correlation signal, but now we know that our target signal exists somewhere around point 17. In a radar detection application, we can use this offset in time (or sample number divided by sampling rate) and the speed of light to determine how far away the detected object is.

Don't let the similarity between convolution and correlation fool you into thinking they are the same thing. Convolution relates an input signal and impulse response to an output signal; whereas correlation allows you to detect a known waveform buried in noise.

# Discrete Fourier Transform (DFT)

Jean Baptiste Joseph Fourier discovered that any continuous waveform can be represented as a sum of sinusoidal waves. This fact also translates into the world of discretes. The DFT takes a discrete input signal and *correlates* it with the sine and cosine basis functions to extract frequency, phase, and amplitude information needed to reconstruct the original signal using only sine and cosine waves. In digital signal processing, we can then use this information to manipulate any input signal.

The DFT can be subdivided into *real* and *complex* versions. The *real* version only deals with real numbers and is the only version that will be considered in this notebook.

The DFT takes an $N$ point input signal and transforms it into two $N/2+1$ point output signals. The two output signals contain the amplitudes of the component sine and cosine waves. These are called the **Real part of $X[]$** and the **Imaginary part of $X[]$**. The values in $ReX[]$ are the amplitudes of the cosine waves, and the values in $ImX[]$ are the amplitudes of the sine waves. Although they are differentiated as "real" and "imaginary", there are no complex numbers in these arrays.

The input signal is usually plotted against the number of samples or against time, so it is said to be in the time domain; whereas the output signals are said to be in the frequency domain (i.e., they are plotted against frequency to show which frequencies in the original signal have a given amplitude).

## DFT Basis Functions

As mentioned before, the DFT takes an input signal and correlates it with the sine and cosine basis functions with unity amplitude. This procedure produces the $ReX[]$ and $ImX[]$ arrays containing the *scaled* amplitudes of each of the basis functions. The basis functions are generated from these equations:
$$
c_k[i] = \cos(2\pi ki/N)
$$
$$
s_k[i] = \sin(2\pi ki/N)
$$
where $c_k[]$ is the cosine wave for the amplitude held in $ReX[k]$, $s_k[]$ is the sine wave for the amplitude held in $ImX[k]$, $i$ is the $i$th point in the input signal, and $N$ is the number of points in the input signal. $k$ is the frequency parameter that completes $k$ cycles in the input signal. For example, $c_5[]$ is the cosine wave that completes five cycles in the $N$ points in $x[]$.

So why are there $N/2+1$ points in the two output signals? If we take the DFT of a 32 point signal in $x[]$, you can only detect 17 discrete sine/cosine waves within that signal. In other words, you cannot detect a signal that completes more than 16 cycles within 32 points (frequencies start at 0 and go up to 16, giving us 17 frequencies).

The plots below show sine and cosine basis function waveforms for 0, 2, 10, and 16 complete cycles for a 32 point signal. These curves are not components of the input signal itself, rather they are 8 of the 34 functions we will correlate the input signal with to extract the amplitudes into $ReX[]$ and $ImX[]$ in the frequency domain. The solid continuous lines are there to help visualize what the actual waveform is doing. The square markers are the actual discrete samples in the basis function that will be correlated with the input signal.

In [4]:
N = 32
N_smooth = 1024
x = np.arange(N)
x_smooth = np.linspace(0, N, N_smooth)

freqs = [0, 2, 10, 16]

sp_titles = []
for f in freqs:
    sp_titles.append(f"c{f}[]")
    sp_titles.append(f"s{f}[]")

fig = make_subplots(rows=4, cols=2, subplot_titles=sp_titles)
for row,freq in enumerate(freqs):
    # Plot the smooth points that show the shape of the basis function
    fig.add_trace(go.Scatter(x=x_smooth,
                             y=np.cos(2*np.pi*freq*x_smooth/N),
                             line=dict(color='black')),
                  row=row+1, col=1)
    fig.add_trace(go.Scatter(x=x_smooth,
                             y=np.sin(2*np.pi*freq*x_smooth/N),
                             line=dict(color='black')),
                  row=row+1, col=2)

    # Plot the discrete markers that show the DFT sample points
    fig.add_trace(go.Scatter(x=x,
                             y=np.cos(2*np.pi*freq*x/N),
                             marker=dict(symbol="square", color='black'),
                             mode="markers"),
                  row=row+1, col=1)

    fig.add_trace(go.Scatter(x=x,
                             y=np.sin(2*np.pi*freq*x/N),
                             marker=dict(symbol="square", color='black'),
                             mode="markers"),
                  row=row+1, col=2)


fig.update_layout(height=900, showlegend=False)
fig.update_xaxes(title_text="Sample number")
fig.update_yaxes(range=[-1.5, 1.5], title_text="Amplitude")
fig.show()

The $c_0[]$ curve is cosine of zero, which is always 1. This means that the $ReX[0]$ array holds the average value of all points in the $x[]$ signal. $s_0[]$ is `sin(0)` which is always zero - no information can be gained from this waveform, and $ImX[0]$ will always be zero.

You can recognize pretty easily that the discrete markers in $c_2[]$ and $s_2[]$ complete 2 cycles even if the continuous waveform underneath wasn't there. These two correspond to $ReX[2]$ and $ImX[2]$.

The discrete points in the 10 cycle waves look a little less recognizable. Even though neither look like sine/cosine waves any longer, 10 complete cycles are still made within the 32 point signal. These two correspond to $ReX[10]$ and $ImX[10]$.

Things get interesting with $c_{N/2}[]$ and $s_{N/2}[]$. The discrete cosine wave is pulling samples from each peak, while the discrete sine wave is sampling at the zero-crossing. If we try to take the samples at a higher rate (i.e. 17 and above), aliasing occurs. What this means is frequencies above the Nyquist frequency, $N/2$, become indistinguishable to the fundamental frequencies. The aliased frequencies are *mirrored* around the Nyquist frequency. To see what I mean, try changing the `16` to a `30` in the `freqs` array above and see what the resulting waveform looks like.

Notice that taking the DFT of a 32 point signal results in two 17 point signals, or 34 total output points. So where did the extra information come from? In reality, there is none - $ImX[N/2]$ and $ImX[0]$ contain zero information, and contribute nothing to the decomposition of the signal $^{[2]}$.

## Analysis - Calculating the DFT

There are a few ways to calculate the DFT. The Fast Fourier Transform (FFT) is the most efficient way to calculate the DFT, but it is somewhat complicated and not very useful when trying to learn what the DFT does to a signal. Here we will calculate the DFT by correlating some input signal with the basis functions.

As mentioned earlier, correlating two signals involves multiplying one signal to another and summing all of the points to find how similar the two signals are. Let's say we have a 64 point sine wave that completes 3 cycles, called $x_1$, and another 64 point signal composed of multiple sine and cosine waves, $x_2$, none of which make three complete cycles. If we correlated both of these signals with the basis function $s_3[]$, we should expect $ImX[3]$ for $x_1$ to contain the average amplitude of $x_1$, and to contain 0 for $x_2$ (since a sine wave completing 3 cycles is not present in $x_2$).

In [5]:
N = 64
samples = np.arange(N)

# Input signal that completes 3 sin cycles
x1 = 1.0*np.sin(2*np.pi*3.0*samples/N)

# Input signal that completes 17 sin cycles, 5 cos cycles, and 9 sin cycles
x2 = 0.1*np.sin(2*np.pi*4.0*samples/N) + 0.5*np.cos(2*np.pi*1.0*samples/N) + 1.4*np.sin(2*np.pi*5.0*samples/N)

# The basis function s3 that we will correlate with the input signals
s3 = np.sin(2*np.pi*3.0*samples/N)

# Correlate the two input signals with the s3 basis function
x1s3 = x1 * s3
x2s3 = x2 * s3

# Print the averages
print(f"Average of x1[] X s3[] = {np.average(x1s3)}")
print(f"Average of x2[] X s3[] = {np.average(x2s3)}")
print(f"Sum of x1[] X s3[] = {np.sum(x1s3)}")
print(f"Sum of x2[] X s3[] = {np.sum(x2s3)}")

fig = make_subplots(rows=3, cols=2, subplot_titles=("x1[], input", "x2[], input",
                                                    "s3[], basis fn", "s3[], basis fn",
                                                    "x1[] X s3[]", "x2[] X s3[]"))

fig.add_trace(go.Scatter(x=samples, y=x1,
                         mode="markers",
                         marker=dict(color='black', symbol="square")),
              row=1, col=1)
fig.add_trace(go.Scatter(x=samples, y=x2,
                         mode="markers",
                         marker=dict(color='black', symbol="square")),
              row=1, col=2)
fig.add_trace(go.Scatter(x=samples, y=s3,
                         mode="markers",
                         marker=dict(color='black', symbol="square")),
              row=2, col=1)
fig.add_trace(go.Scatter(x=samples, y=s3,
                         mode="markers",
                         marker=dict(color='black', symbol="square")),
              row=2, col=2)
fig.add_trace(go.Scatter(x=samples, y=x1s3,
                         mode="markers",
                         marker=dict(color='black', symbol="square")),
              row=3, col=1)
fig.add_trace(go.Scatter(x=samples, y=x2s3,
                         mode="markers",
                         marker=dict(color='black', symbol="square")),
              row=3, col=2)

fig.update_layout(height=800, showlegend=False)
fig.update_xaxes(title_text="Sample number")
fig.update_yaxes(title_text="Amplitude", range=[-2.3, 2.3])
fig.show()

Average of x1[] X s3[] = 0.5
Average of x2[] X s3[] = 1.3877787807814457e-17
Sum of x1[] X s3[] = 32.0
Sum of x2[] X s3[] = 8.881784197001252e-16


The sum (and average) of $x_2[] \times s_3[]$ is zero, indicating the target waveform was not present in $x_2$.

Is it just a coincidence that the sum of $x_1[]\times s_3[]$ happens to be $N/2$? Correlating the input signal with the basis function is essentially performing an inner product of the two arrays. When multiplying these signals element-wise, you get:
$$
y[n] = x_1[n]\times s_3[n] = A*\sin(2\pi kn/N) * \sin(2\pi kn/N)
$$
$$
y[n] = A*\sin^2 (2\pi kn/N)
$$
with $A$ being the amplitude of $x_1$ and $k=3$. It is a property of a $sin^2(x)$ and $cos^2(x)$ that they average to 0.5 for any (complete) number of cycles. That means that each point in $y[n]$, on average, contributes 0.5 to the sum, and we can replace $\sum sin^2$ with $\mu*N$:
$$
\mu= \frac{1}{N}\sum_{n=1}^{N}x[n] \to \sum x[n] = \mu*N
$$
$$
ImX[3] = \sum_{n=0}^{N-1}y[n] = \sum_{n=0}^{N-1}A*\sin^2(2\pi kn/N) = A * N * 0.5
$$
Since $x_1$ has an amplitude of 1, the sum comes out to $N*0.5 = 32$. So it's not necessarily a coincidence - had our amplitude been 2, the sum would come out to be $N$; if $A=3$, $ImX[3]=\frac{3N}{2}$, etc.

This creates an interesting consequence when plotting $ReX[]$ and $ImX[]$. If we pass a 32 point signal with a single arbitrary frequency and an amplitude of 10, then plotting that frequency will tell us the amplitude was 160. This means we need to scale the amplitudes to retrieve the sinusoidal amplitudes:
$$
Re\bar X[k] = \frac{ReX[k]}{N/2}
$$
$$
Im\bar X[k] = -\frac{ImX[k]}{N/2}
$$
except for two special cases:
$$
Re\bar X[0] = \frac{ReX[0]}{N}
$$
$$
Re\bar X[N/2] = \frac{ReX[N/2]}{N}
$$
The reason the first and last indices of the real part of X are scaled differently is because the frequency domain is defined as a **spectral density**$^{[3]}$. Each sample in the frequency domain is an amplitude for a discrete bin in the total frequency bandwidth. We know that the total frequency bandwidth ranges from [0, N/2]. This means that each bin's width is $\frac{1}{N/2}$, or $2/N$ (with the exception of the first and last indices, since the amplitudes lie in the middle of each band).

<center><img src="https://github.com/jbucklin/colab/blob/main/DSP/images/spectral_density.png?raw=1">$^{[3]}$</center>

To sum all of this up (pun intended), we can generalize this algorithm of correlating the input signal with the basis functions of each frequency ($c_k[]$ and $s_k[]$) for all of the other points in the $ReX[]$ and $ImX[]$ arrays:
$$
ReX[k] = \sum_{i=0}^{N-1}x[i] \cos(2\pi ki/N)
$$
$$
ImX[k] = -\sum_{i=0}^{N-1}x[i] \sin(2\pi ki/N)
$$
These are the DFT analysis equations, and they give us the amplitudes of the sin and cos waves that exist in $x[i]$ in the *frequency* domain. Apply the scaling factor described above to normalize the amplitudes. The minus sign for $ImX[]$ is to keep the *real* DFT consistent with the *complex* DFT $^{[4]}$. See Chapter 29 of the reference book for more details.

The code below creates an input signal with three arbitrarily selected frequencies, correlates them with the DFT basis functions, and plots the normalized amplitudes:

In [7]:
"""
Correlate the signal with the basis functions to find
the ReX[] and ImX[] amplitude arrays in the frequency
domain. Then calculate the magnitude and phase of X[].
"""
def calc_DFT(x):
    N = len(x)
    ReX = np.zeros(int(N/2)+1)
    ImX = np.zeros(int(N/2)+1)
    for k in range(int(N/2)+1):
        ReX[k] =  sum([x[i]*np.cos(2*np.pi*k*i/N) for i in range(len(x))])
        ImX[k] = -sum([x[i]*np.sin(2*np.pi*k*i/N) for i in range(len(x))])

    # Scale the amplitudes
    ReX /= (N/2)
    ReX[0] /= 2
    ReX[int(N/2)] /= 2
    ImX /= -(N/2)

    # Calculate the magnitude and phase of X
    MagX = np.sqrt(ReX**2 + ImX**2)
    PhaseX = 180/np.pi * np.arctan2(ImX, ReX)

    # Create plots
    fig = make_subplots(rows=3, cols=2,
                        specs=[[{"colspan": 2}, None],
                               [{}, {}],
                               [{}, {}]],
                        subplot_titles=("x[], Input Signal",
                                        "ReX[]", "ImX[]",
                                        "Magnitude of X", "Phase of X"))
    fig.add_trace(go.Scatter(y=x,
                             mode="lines+markers",
                             marker=dict(symbol="square")),
                  row=1, col=1)
    fig.add_trace(go.Scatter(y=ReX,
                             mode="lines+markers",
                             marker=dict(symbol="square")),
                  row=2, col=1)
    fig.add_trace(go.Scatter(y=ImX,
                             mode="lines+markers",
                             marker=dict(symbol="square")),
                  row=2, col=2)
    fig.add_trace(go.Scatter(y=MagX,
                             mode="lines+markers",
                             marker=dict(symbol="square")),
                  row=3, col=1)
    fig.add_trace(go.Scatter(y=PhaseX,
                             mode="lines",
                             marker=dict(symbol="square")),
                  row=3, col=2)

    fig.update_xaxes(title_text="Sample Number", row=1, col=1)
    fig.update_xaxes(title_text="Frequency Sample Number", row=2)
    fig.update_xaxes(title_text="Frequency Sample Number", row=3)
    fig.update_yaxes(title_text="Amplitude")
    fig.update_yaxes(title_text="Phase (deg)", row=3, col=2)
    fig.update_layout(height=700, showlegend=False)
    fig.show()

# Create a 1024 signal with various frequencies
N = 1024
samples = np.arange(N)
x = 5.0*np.sin(2*np.pi*42.0*samples/N) + 2.5*np.cos(2*np.pi*19.0*samples/N) + np.sin(2*np.pi*4.0*samples/N)
calc_DFT(x)

The input signal is a sum of three sinusoidal waves: a cosine wave with an amplitude of 2.5 and frequency of 19, a sine wave with an amplitude of 1 and frequency of 4, and a sine wave with an amplitude of 5 and frequency 42. You can read that from the code, or you can look at the $ReX[]$ and $ImX[]$ plots above. $ReX[]$ shows the *scaled* amplitudes of all cosine waves found in the input signal, and likewise $ImX[]$ for the sine waves.

$ReX[]$ and $ImX[]$ are referred to as **rectangular notation**. For simple signals like $x[]$ above with only a few different frequencies, rectangular notation can be sufficient to see what's going on with the signal. This is not the case in most practical situations, and converting $ReX[]$ and $ImX[]$ to **polar notation** can paint a better picture of the signal. Polar notation gives us the magnitude of $ReX[]$ and $ImX[]$ as well as the phase for each frequency.

<center><img src="https://github.com/jbucklin/colab/blob/main/DSP/images/polar_notation.png?raw=1">$^{[5]}$</center>

To convert to polar notation:
$$
MagX[k] = \sqrt{ReX[k]^2 + ImX[k]^2}
$$
$$
PhaseX[k] = \arctan(ImX[k] / ReX[k])
$$
The magnitude and the phase contain the same information as $ReX[]$ and $ImX[]$, just in a different representation. You can also convert back to rectangular notation from polar:
$$
ReX[k] = MagX[k] \cos(PhaseX[k])
$$
$$
ImX[k] = MagX[k] \sin(PhaseX[k])
$$

There can be some nuisances when converting to polar notation:
- When doing this in code, ensure to use the `atan2` function instead of `atan` to get phase values for all 4 quadrants of the plane. This is done in the code above.
- For frequencies with very small amplitudes, the phase can look erratic and not very informative. This is exactly what's happening in the phase plot above, since there are only 3 out of the 513 possible frequencies with nonzero amplitudes. If you zoom in on the 3 frequencies in the phase plot, you'll find 90 degrees for the sine components, and 0 degrees for the cosine components.

Let's look at a more complicated signal:

In [8]:
# Create a 512 point signal with nonzero amplitudes for
# the first 100 frequencies, while causing the amplitudes
# to also oscillate
N = 512
samples = np.arange(N)
x = np.zeros(N)
for k in range(100):
    A = np.sin(2*np.pi*2.0*k/100)
    B = np.cos(2*np.pi*2.0*k/100)
    x += A*np.sin(2*np.pi*k*samples/N ) + B*np.cos(2*np.pi*k*samples/N)

calc_DFT(x)

The amplitudes of the sine and cosine waves oscillate, making it a bit more difficult to understand the rectangular notation. The polar notation on the other hand tells us that we have unity amplitude for frequencies between 0-99. The phase tells us where a particular frequency is located in the polar plane with respect to the basis frequency. Notice that the phase once again becomes erratic once the magnitude becomes zero.

## Synthesis - Calculating the Inverse DFT

Given $Re\bar X[]$ and $Im\bar X[]$ (or similarly $MagX[]$ and $PhaseX[]$), we can calculate the **inverse DFT** to recreate the original signal:
$$
x[i] = \sum_{k=o}^{N/2} Re\bar X[k] \cos(2\pi ki/N) + \sum_{k=0}^{N/2} Im\bar X[k] \sin(2\pi ki/N)
$$
If given $ReX[]$ and $ImX[]$, you must normalize the amplitudes before snythesizing the signal.

# Summary

Given some input signal, we can correlate that signal with the DFT basis functions to extract frequency information. Correlating the input signal with the basis functions gives us amplitudes of the sine and cosine components in the frequency domain. We can then scale these amplitudes to retrieve the sinusoidal amplitudes in the original signal. The amplitude arrays can then be converted into polar notation for a more informative representation.

The DFT analysis and synthesis equations described here are not the only (or most efficient) ways to calculate the DFT and inverse DFT. In almost all cases, you would be using the FFT from some software library (`numpy.fft`). However, in learning what happens to a signal when performing the transform, the DFT analysis equation is probably the best way to understand the process.

# References

[1] Smith, Steven W. (1999). Ch 7 Properties of Convolution. *Guide to Digital Signal Processing* (2nd ed., pp 138)\
[2] Smith, Steven W. (1999). Ch 8 The Discrete Fourier Transform. *Guide to Digital Signal Processing* (2nd ed., pp 152)\
[3] Smith, Steven W. (1999). Ch 8 The Discrete Fourier Transform. *Guide to Digital Signal Processing* (2nd ed., pp 156)\
[4] Smith, Steven W. (1999). Ch 8 The Discrete Fourier Transform. *Guide to Digital Signal Processing* (2nd ed., pp 158)\
[5] Smith, Steven W. (1999). Ch 8 The Discrete Fourier Transform. *Guide to Digital Signal Processing* (2nd ed., pp 162)
